<a href="https://colab.research.google.com/github/akitim/comptech-winter-school-2022_covid19-prediction/blob/main/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Копия prepare_data в МОЕМ репозитории 

In [21]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_rows', 200)

In [2]:
!git clone https://github.com/comptech-winter-school/covid19-prediction

Cloning into 'covid19-prediction'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 208 (delta 96), reused 79 (delta 22), pack-reused 0
Receiving objects: 100% (208/208), 96.10 MiB | 24.32 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Checking out files: 100% (14/14), done.


In [3]:
path = "covid19-prediction"
all_files = Path(path).glob("data*.csv") 

df = pd.concat(map(pd.read_csv, all_files));

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
# all_files = Path(path).glob("*.csv") 

# df = pd.concat(map(pd.read_csv, all_files))

In [4]:
df.shape

(2637010, 22)

In [5]:
df.head()

,Unnamed: 0,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio,Province/State,Country/Region,Last Update,Latitude,Longitude
0,0,45001.0,Abbeville,South Carolina,US,2020-04-29 02:32:29,34.223334,-82.461707,29.0,0.0,0.0,29.0,"Abbeville, South Carolina, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,22001.0,Acadia,Louisiana,US,2020-04-29 02:32:29,30.295065,-92.414197,130.0,9.0,0.0,121.0,"Acadia, Louisiana, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,51001.0,Accomack,Virginia,US,2020-04-29 02:32:29,37.767072,-75.632346,223.0,4.0,0.0,219.0,"Accomack, Virginia, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,16001.0,Ada,Idaho,US,2020-04-29 02:32:29,43.452658,-116.241552,662.0,16.0,0.0,646.0,"Ada, Idaho, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,19001.0,Adair,Iowa,US,2020-04-29 02:32:29,41.330756,-94.471059,1.0,0.0,0.0,1.0,"Adair, Iowa, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def merge_columns(f1, f2, df):
    d = df.loc[:, [f1, f2]].copy().reset_index()
    d.loc[d[f1].isna(), f1] = d.loc[d[f1].isna(), f2]
    return d[f1]

In [7]:
# убираем страны c подтвержденными случаями < 10^5
def remove_small_countries(df):
    countries = df.groupby("Country_Region").last().query('Confirmed > 100000').index.to_list()
    return df[df.Country_Region.isin(countries)]

In [8]:
def report_shape(df):
    print(df.shape)
    return df

In [22]:
!mkdir tmp
data=(df
 .assign(Country_Region = merge_columns("Country_Region", "Country/Region", df))
 .assign(Last_Update = merge_columns("Last_Update", "Last Update", df))
 .assign(Last_Update = pd.to_datetime(df["Last_Update"]).dt.date)
 .loc[:, ["Country_Region", "Last_Update", "Confirmed"]]
#  .assign(New_Cases = df['']("Last_Update", "Last Update", df))
 .dropna()
 .groupby(["Country_Region", "Last_Update"])
 .sum()
 .reset_index()
 .set_index("Last_Update")
#  .sort_index()
#  .pipe(remove_small_countries)
#  .pipe(report_shape)
#  .to_parquet("tmp/data.parquet")
)

mkdir: cannot create directory ‘tmp’: File exists


In [12]:
data['New_Cases'] = 0
data['Confirmed'] = data['Confirmed'].shift(1)

In [16]:
data[data['Country_Region'].isin(['Afghanistan', 'Albania'])]

,Country_Region,Confirmed,New_Cases
Last_Update,,,
2020-03-23,Afghanistan,NaN,0
2020-03-24,Afghanistan,5.0,0
2020-03-26,Afghanistan,0.0,0
2020-03-27,Afghanistan,0.0,0
2020-03-30,Afghanistan,0.0,0
...,...,...,...
2022-01-20,Albania,241368.0,0
2022-01-21,Albania,247696.0,0
2022-01-22,Albania,308067.0,0


In [29]:
# display(data)
temp_df, df_ = pd.DataFrame(), pd.DataFrame()
for country in data['Country_Region'].unique():
    temp_df = data.loc[data['Country_Region'] == country, :].copy()
    temp_df.loc['New_Cases'] = (temp_df['Confirmed'] - temp_df['Confirmed'].shift(1)).dropna()
    df_ = pd.concat([df_, temp_df], axis=0)



In [27]:
any(data['Confirmed'].isna())

False

In [ ]:
def df_nans_calc(df):
    valid_se = (~df.isna()).sum(axis=0)
    df_nans = pd.concat([valid_se, df.shape[0] - valid_se], axis=1)
    df_nans.columns = ['valid', 'NaNs']
    return df_nans

df_nans_calc(data)

,valid,NaNs
Unnamed: 0,2637010,0
FIPS,2163669,473341
Admin2,2167116,469894
Province_State,2510659,126351
Country_Region,2637010,0
Last_Update,2637010,0
Lat,2571664,65346
Long_,2571664,65346
Confirmed,2636982,28
Deaths,2636576,434


In [ ]:
df['Country_Region'] = merge_columns('Country_Region', 'Country/Region', df).isna().sum()
pd.isna(df['Country_Region']).sum()

0